In [2]:
# import necessary libaries
import pandas as pd
import requests
import os
from dotenv import load_dotenv
%load_ext autoreload

%autoreload 2

import sqlite3

import matplotlib.pyplot as plt


In [3]:
# Import `AlphaVantageAPI`
from data import AlphavantageApi


# Create instance of `AlphaVantageAPI` class
av = AlphavantageApi()

print("av type:", type(av))

av type: <class 'data.AlphavantageApi'>


In [8]:
# Define Suzlon ticker symbol
ticker = "SUZLON.BSE"
# Use your `av` object to get daily data
df_suzlon = av.get_daily(ticker=ticker)

print("df_suzlon type:", type(df_suzlon))
print("df_suzlon shape:", df_suzlon.shape)
df_suzlon.head()

df_suzlon type: <class 'pandas.core.frame.DataFrame'>
df_suzlon shape: (100, 5)


,open,high,low,close,volume
date,,,,,
2026-01-12,49.21,49.40,47.90,49.21,11702771.0
2026-01-09,50.59,51.07,48.89,49.20,13631320.0
2026-01-08,52.79,53.40,50.69,50.93,7844493.0
2026-01-07,53.08,53.70,52.67,52.91,4056092.0
2026-01-06,53.65,53.99,52.86,53.45,3749914.0


In [4]:
#create sqlite connection

connection = sqlite3.connect(database=os.getenv("db_name"), check_same_thread=False)

print("connection type:", type(connection))

connection type: <class 'sqlite3.Connection'>


In [5]:
# Import class definition
from data import SQLRepository

# Create instance of class
repo = SQLRepository(connection=connection)

# Does `repo` have a "connection" attribute?
assert hasattr(repo, "connection")

# Is the "connection" attribute a SQLite `Connection`?
assert isinstance(repo.connection , sqlite3.Connection)

In [9]:
ticker = "SUZLON.BSE"
response = repo.insert_table(table_name=ticker, records=df_suzlon, if_exists="replace")

# Does your method return a dictionary?
assert isinstance(response, dict)

# Are the keys of that dictionary correct?
assert sorted(list(response.keys())) == ["records_inserted","transaction_successful"]

In [10]:
%load_ext sql
%sql sqlite:///stocks.sqlite


In [11]:
%%sql
SELECT *
FROM 'SUZLON.BSE'
LIMIT 5

 * sqlite:///stocks.sqlite
Done.


date,open,high,low,close,volume
2026-01-12 00:00:00,49.21,49.4,47.9,49.21,11702771.0
2026-01-09 00:00:00,50.59,51.07,48.89,49.2,13631320.0
2026-01-08 00:00:00,52.79,53.4,50.69,50.93,7844493.0
2026-01-07 00:00:00,53.08,53.7,52.67,52.91,4056092.0
2026-01-06 00:00:00,53.65,53.99,52.86,53.45,3749914.0


In [20]:
sql = "SELECT * FROM 'SUZLON.BSE'"
df_suzlon_test = pd.read_sql(
    sql=sql, con=connection, parse_dates=["date"], index_col="date"
)
#pd.read_sql( sql=sql, con=connection, parse_dates=["date"], index_col="date")
print("df_suzlon_test type:", type(df_suzlon_test))
print()
print(df_suzlon_test.info())
df_suzlon_test.head()

df_suzlon_test type: <class 'pandas.core.frame.DataFrame'>

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100 entries, 2026-01-12 to 2025-08-25
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    100 non-null    float64
 1   high    100 non-null    float64
 2   low     100 non-null    float64
 3   close   100 non-null    float64
 4   volume  100 non-null    float64
dtypes: float64(5)
memory usage: 4.7 KB
None


,open,high,low,close,volume
date,,,,,
2026-01-12,49.21,49.40,47.90,49.21,11702771.0
2026-01-09,50.59,51.07,48.89,49.20,13631320.0
2026-01-08,52.79,53.40,50.69,50.93,7844493.0
2026-01-07,53.08,53.70,52.67,52.91,4056092.0
2026-01-06,53.65,53.99,52.86,53.45,3749914.0


In [21]:
df_suzlon.columns

Index(['open', 'high', 'low', 'close', 'volume'], dtype='object')

In [24]:
# Assign `read_table` output to `df_suzlon`
df_suzlon = repo.read_table(table_name="SUZLON.BSE", limit=2500)  # noQA F821

# Is `df_suzlon` a DataFrame?
assert isinstance(df_suzlon, pd.DataFrame)

# Does it have a `DatetimeIndex`?

assert isinstance(df_suzlon.index, pd.DatetimeIndex)
# Is the index named "date"?
assert df_suzlon.index.name == "date"

# Does it have 2,500 rows and 5 columns?
assert df_suzlon.shape == (100, 5)

# Are the column names correct?
assert df_suzlon.columns.to_list() == ['open', 'high', 'low', 'close', 'volume']

# Are the column data types correct?
assert all(df_suzlon.dtypes == float)

# Print `df_suzlon` info
print("df_suzlon shape:", df_suzlon.shape)
print()
print(df_suzlon.info())
df_suzlon.head()

df_suzlon shape: (100, 5)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100 entries, 2026-01-12 to 2025-08-25
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    100 non-null    float64
 1   high    100 non-null    float64
 2   low     100 non-null    float64
 3   close   100 non-null    float64
 4   volume  100 non-null    float64
dtypes: float64(5)
memory usage: 4.7 KB
None


,open,high,low,close,volume
date,,,,,
2026-01-12,49.21,49.40,47.90,49.21,11702771.0
2026-01-09,50.59,51.07,48.89,49.20,13631320.0
2026-01-08,52.79,53.40,50.69,50.93,7844493.0
2026-01-07,53.08,53.70,52.67,52.91,4056092.0
2026-01-06,53.65,53.99,52.86,53.45,3749914.0


In [17]:
def read_table(table_name, limit=None):
    """Read table from database.

    Parameters
    ----------
    table_name : str
        Name of table in SQLite database.
    limit : int, None, optional
        Number of most recent records to retrieve. If `None`, all
        records are retrieved. By default, `None`.

    Returns
    -------
    pd.DataFrame
        Index is DatetimeIndex "date". Columns are 'open', 'high',
        'low', 'close', and 'volume'. All columns are numeric.
    """
    # Create SQL query (with optional limit)
    if limit:
        sql = f"SELECT * FROM '{table_name}' LIMIT {limit}"
    else:
        sql = f"SELECT * FROM '{table_name}'"
    

    # Retrieve data, read into DataFrame
    df = pd.read_sql(
    sql=sql, con=connection, parse_dates=["date"], index_col="date"
)

    # Return DataFrame
    return df